In [ ]:
from lmfit import Parameters
from lmfit import minimize, fit_report

In [ ]:
import pyroll.core as pr
import pyroll.pillar_model
import pyroll.neutral_point_estimator
import pyroll.sparling_spreading
import pyroll.hensel_force_torque
import pyroll.freiberg_flow_stress
import pyroll.ring_model_thermal

In [ ]:
from in_profile import create_in_profile_from_surface_temperature
from rolling_sequence import rolling_train_e112, rolling_train_e107
from utilities import *

In [ ]:
pyroll.pillar_model.Config.PILLAR_COUNT = 30
pyroll.neutral_point_estimator.Config.ESTIMATOR = "EQUAL"
ROLL_SURFACE_TEMPERATURE = 50 + 273.15
DISK_ELEMENT_COUNT = 30

In [ ]:
@pr.CoolingPipe.DiskElement.heat_transfer_coefficient
def heat_transfer_coefficient_model_moreales(self: pr.CoolingPipe.DiskElement):
    mean_surface_temperature = (self.in_profile.surface_temperature + self.out_profile.surface_temperature) / 2
    profile_surface_temperature_celsius = mean_surface_temperature - 273.15
    return 13950 + 5.12 * 1e6 / profile_surface_temperature_celsius

In [ ]:
in_profile = create_in_profile_from_surface_temperature(1050 + 273.15)
in_profile.ring_temperatures = in_profile.ring_temperatures

In [ ]:
rolling_train_e112 = rolling_train_e112(ROLL_SURFACE_TEMPERATURE, DISK_ELEMENT_COUNT)
rolling_train_e107 = rolling_train_e107(ROLL_SURFACE_TEMPERATURE, DISK_ELEMENT_COUNT)

In [ ]:
rolling_train_e112.solve(in_profile)

In [ ]:
rolling_train_e107.solve(in_profile)

In [ ]:
def solve_total_wear_depths_as_contour_line_with_constant(params):
    sequences = [
        {
            "label": "e11_2",
            "sequence": rolling_train_e112
        },
        {
            "label": "e10_7",
            "sequence": rolling_train_e107
        }
    ]

    area_errors = []
    wear_coefficient = params['wear_coefficient'].value

    for entry in sequences:
        groove_label = entry["label"]
        sequence = entry["sequence"]
        roll_passes = sequence.roll_passes
        measurements = extract_wear_contours_from_measurement(groove_label)


        for measurement_id, labels, tonnages, measured_wear_contours in measurements:
            for roll_pass in roll_passes:
                for label, tonnage, measured_wear_contour in zip(labels, tonnages, measured_wear_contours):
                    if roll_pass.label == label:

                        calculated_wear_contour = calculate_wear_contour(roll_pass, wear_coefficient, tonnage * 1000)
                        area_error = calculate_area_between_contours(calculated_wear_contour, measured_wear_contour)
                        area_errors.append(area_error)
                    else:
                        continue
    rms_error = root_mean_square_value(np.array(area_errors))
    return rms_error


In [ ]:
def solve_total_wear_depths_as_contour_line_with_function(params):
    sequences = [
        {
            "label": "e11_2",
            "sequence": rolling_train_e112
        },
        {
            "label": "e10_7",
            "sequence": rolling_train_e107
        }
    ]

    area_errors = []
    base_wear_coefficient = params['base_wear_coefficient'].value
    roll_gap_ratio_exponent = params['roll_gap_ratio_exponent'].value

    for entry in sequences:
        groove_label = entry["label"]
        sequence = entry["sequence"]
        roll_passes = sequence.roll_passes
        measurements = extract_wear_contours_from_measurement(groove_label)


        for measurement_id, labels, tonnages, measured_wear_contours in measurements:
            for roll_pass in roll_passes:
                for label, tonnage, measured_wear_contour in zip(labels, tonnages, measured_wear_contours):
                    if roll_pass.label == label:

                        calculated_wear_contour = calculate_wear_contour_as_function(roll_pass, base_wear_coefficient, roll_gap_ratio_exponent, tonnage * 1000)
                        area_error = calculate_area_between_contours(calculated_wear_contour, measured_wear_contour)
                        area_errors.append(area_error)
                    else:
                        continue
    rms_error = root_mean_square_value(np.array(area_errors))
    return rms_error


In [ ]:
params_constant_cobyla = Parameters()
params_constant_cobyla.add('wear_coefficient', value=1e-6, min=1e-10, max=1e-3, vary=True)

params_function_cobyla = Parameters()
params_function_cobyla.add('base_wear_coefficient', value=1e-6, min=1e-10, max=1e-3, vary=True)
params_function_cobyla.add('roll_gap_ratio_exponent', value=1e-6, min=1e-10, max=1e-3, vary=True)

In [ ]:
result_constant_cobyla = minimize(
    solve_total_wear_depths_as_contour_line_with_constant,
    params_constant_cobyla,
    nan_policy='omit',
    max_nfev=100,
    method="cobyla"
)

result_function_cobyla = minimize(
    solve_total_wear_depths_as_contour_line_with_function,
    params_function_cobyla,
    nan_policy='omit',
    max_nfev=100,
    method="cobyla"
)


result_constant_ls = minimize(
    solve_total_wear_depths_as_contour_line_with_constant,
    params_constant_cobyla,
    nan_policy='omit',
    max_nfev=100,
    method="least_squares"
)

result_function_ls = minimize(
    solve_total_wear_depths_as_contour_line_with_function,
    params_function_cobyla,
    nan_policy='omit',
    max_nfev=100,
    method="least_squares"
)

In [ ]:
print(fit_report(result_constant_cobyla))

In [ ]:
print(fit_report(result_function_cobyla))

In [ ]:
print(fit_report(result_constant_ls))

In [ ]:
print(fit_report(result_function_ls))